In [5]:
!conda install pandas==1.3.4
!conda install numpy

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / ^C
failed with repodata from current_repodata.json, will retry with next repodata source.

CondaError: KeyboardInterrupt

^C

CondaError: KeyboardInterrupt



In [2]:
%load_ext autoreload
%autoreload 2
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cuda', index=0)

In [ ]:
!pip install pandas
!pip install torch
!pip install pycombat
!pip install scikit-learn

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.autograd import Variable

/nobackup/users/razgaon/projects/condas/cytof/miniconda3/envs/cytof/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
# Engaging
# data_dir = f'/home/razgaon/ondemand/data/sys/myjobs/projects/default/3/'
# Satori
data_dir = f'/home/razgaon/razgaon/'
NSCLC_pickle = f'{data_dir}NSCLC_prepared_data_test.pickle'
PBMC_pickle = f'{data_dir}PBMC_prepared_data_test.pickle'
NSCLC = pd.read_pickle(NSCLC_pickle)
PBMC = pd.read_pickle(PBMC_pickle)


# Try to add an id that maps to the fcs file name


In [74]:
PBMC['name']

KeyError: 'name'

In [71]:
PBMC_df = PBMC.sample(100000)
NSCLC_df = NSCLC.sample(100000)

In [ ]:
#check if pycombat requires normalization first for each dataset, or after
#concat (keep all markers), batch normalize on X with NaNs, umap to confirm quality,  min-max scale, then vae, umap to confirm quality

In [61]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
excluded_markers = ['id', 'Time', 'Eventlength', 'sample', 'batch']

PBMC_markers = [col for col in PBMC_df.columns if col not in excluded_markers]
PBMC_df[PBMC_markers] = scaler.fit_transform(PBMC_df[PBMC_markers])
PBMC_df['source'] = 0

NSCLC_markers = [col for col in NSCLC_df.columns if col not in excluded_markers]
NSCLC_df[NSCLC_markers] = scaler.fit_transform(NSCLC_df[NSCLC_markers])
NSCLC_df['source'] = 1

In [63]:
df = pd.concat([PBMC_df, NSCLC_df])
df
# df.to_csv('original_df.csv')

,id,CD11b,Beads,CD80,FoxP3,CD33,CXCR5,CCR6,Gata3,RORgT,...,CCR7,Er168Di,CD38,GITR,CD56,Yb176Di,Os189Di,DNA1,DNA2,Cisplatin
785943,785944,0.000000,0.00000,0.000000,0.485537,0.782830,0.000000,0.042640,0.000000,0.204006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
745519,745520,0.000000,0.00000,0.000000,0.128126,0.000000,0.000000,0.122072,0.345791,0.204006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51647,51648,0.000000,0.00000,0.000000,0.294445,0.167676,0.682063,0.496258,0.345791,0.235161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1110090,1110091,0.000000,0.10096,0.069203,0.505993,0.000000,0.000000,0.000000,0.301696,0.090279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
748812,748813,0.068608,0.00000,0.000000,0.218417,0.000000,0.000000,0.000000,0.039188,0.090279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46675,46676,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.079267,0.061541,0.0,0.321914,0.218055,0.131698
214377,214378,NaN,NaN,NaN,NaN,NaN,0.000000,0.028004,NaN,NaN,...,0.050934,0.000000,0.083122,0.000000,0.021496,0.042198,0.0,0.430598,0.346291,0.050559
122752,122753,NaN,NaN,NaN,NaN,NaN,0.000000,0.054973,NaN,NaN,...,0.074282,0.040897,0.314993,0.036253,0.061541,0.079267,0.0,0.376180,0.271845,0.114250
101304,101305,NaN,NaN,NaN,NaN,NaN,0.000000,0.080172,NaN,NaN,...,0.291318,0.000000,0.022542,0.000000,0.042198,0.042198,0.0,0.343310,0.196315,0.000000


In [48]:

class MyDataset(Dataset):
 
    def __init__(self,df):
        x=df.values
        self.x_train=torch.tensor(x,dtype=torch.float32)

    def __len__(self):
        return len(self.x_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx]


In [33]:
# Convert metals to their marker names to ensure consistency when merging dataframes
overlap = list(set(PBMC_markers).intersection(set(NSCLC_markers)))

In [49]:
# markers = NSCLC_markers + PBMC_markers
dataset = MyDataset(df[overlap])
# # Data loader
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset, batch_size=1000, shuffle=True)


In [43]:
class VAE(nn.Module):
    def __init__(self, x_dim, latent_dim=5, hdim=50):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.fc1 = nn.Linear(x_dim , hdim)
        self.fc21 = nn.Linear(hdim, latent_dim)
        self.fc22 = nn.Linear(hdim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hdim)
        self.fc4 = nn.Linear(hdim, x_dim)
        
    def encode(self, x):
        x_ = torch.cat([x], dim=-1)
        h1 = F.relu(self.fc1(x_))
        return self.fc21(h1), self.fc22(h1)
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)
    def reparameterize(self, mu, var):
#         if self.distance == 'standard':
        if self.training:
            std = var.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            result = eps.mul(std).add_(mu)
            return result
        else:
            return mu
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.x_dim))
        latent = self.reparameterize(mu, logvar)
        reconstructed = self.decode(latent)
        return reconstructed, latent

In [44]:
def compute_kernel(x, y, device):
    x_size = x.to(device).size(0)
    y_size = y.to(device).size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim).to(device)
    tiled_y = y.expand(x_size, y_size, dim).to(device)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input)

def compute_mmd(x, y, device):
    x_kernel = compute_kernel(x, x, device)
    y_kernel = compute_kernel(y, y, device)
    xy_kernel = compute_kernel(x, y, device)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

def vae_loss_function(inputs, encoded, decoded, true_samples, device):
    #reconstruction loss - decoding
    nll = (decoded - inputs).pow(2).mean()
    #distance + reconstruction loss
    mmd = compute_mmd(true_samples, encoded.to(device), device)
    return(nll, mmd)

In [45]:
def train(net, learning_rate, epochs, train_loader, test_loader, optimizer = 'Adam'):
    if optimizer == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr = learning_rate)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr = learning_rate, 
                              momentum = .95, nesterov = True)
    
    for epoch in range(epochs):
        training_loss = 0
        training_reconstruction_error = 0
        training_mmd = 0

        net.train()
        for batchnum, X in enumerate(train_loader):
            optimizer.zero_grad()
            reconstruction, mu = net(X)
            true_samples = torch.randn(mu.shape)
            reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)
            loss = reconstruction_error + mmd
            loss.backward()
            
            optimizer.step()

            training_reconstruction_error += reconstruction_error
            training_mmd  += mmd
            training_loss += loss

        training_reconstruction_error /= (batchnum+1)
        training_mmd  /= (batchnum+1)
        training_loss /= (batchnum+1)
        print('Training loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, training_loss, training_reconstruction_error, training_mmd) )
        
        #Testing loop

        testing_reconstruction_error = 0
        testing_mmd = 0

        with torch.no_grad():
            for batchnum, X in enumerate(test_loader):
                reconstruction, mu = net(X)
                true_samples = torch.randn(mu.shape)
                reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)

                testing_reconstruction_error += reconstruction_error
                testing_mmd += mmd
            
            testing_reconstruction_error /= (batchnum+1)
            testing_mmd /= (batchnum+1)
            print('Testing loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, testing_reconstruction_error + testing_mmd, testing_reconstruction_error, testing_mmd) )
        

In [19]:
net = VAE(x_dim=dataset.x_train.shape[1])

In [20]:
train(net, learning_rate = .0001, epochs = 30,
      train_loader = train_loader, test_loader = test_loader)

Training loss for epoch 0 is 0.10110502, Reconstruction is 0.09657765, mmd is 0.00452731


KeyboardInterrupt: 

In [36]:
torch.save(net, 'model_state')

In [10]:
# Model class must be defined somewhere
model = torch.load('model_state')
model.eval()

VAE(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc21): Linear(in_features=50, out_features=5, bias=True)
  (fc22): Linear(in_features=50, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=13, bias=True)
)

In [47]:
dataset.x_train

tensor([[0.1200, 0.0000, 0.0334,  ..., 0.6372, 0.6633, 0.0000],
        [0.5676, 0.0000, 0.2571,  ..., 0.7047, 0.3044, 0.0484],
        [0.1200, 0.0000, 0.0656,  ..., 0.5747, 0.3217, 0.0000],
        ...,
        [0.5077, 0.2861, 0.0000,  ..., 0.0000, 0.2371, 0.2309],
        [0.3052, 0.3126, 0.0000,  ..., 0.0000, 0.2145, 0.2309],
        [0.3496, 0.3704, 0.1016,  ..., 0.5858, 0.0696, 0.4264]],
       dtype=torch.float64)

In [50]:
r, latent = model(dataset.x_train)

In [56]:
latent_df = pd.DataFrame(latent.detach().cpu().numpy(), index = df.index)
latent_df['source'] = df.source

In [57]:
latent_df.to_pickle('latent.pickle')

In [58]:
latent.shape

torch.Size([200000, 5])

In [59]:
x_train = pd.DataFrame(dataset.x_train.detach().cpu().numpy(), index = df.index)
x_train['source'] = df.source
x_train.to_pickle('x_train.pickle')